In [7]:
! pip install opencv-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
! cmake -D WITH_WIN32UI=ON -D WITH_GTK=ON -D WITH_COCOA=ON


CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


CMake Error: The source directory "E:/ICSDG_HACKATHON" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.


In [8]:
import cv2
import numpy as np
import math

class ObjectDetection:
    def __init__(self, weights_path="yolov4.weights", cfg_path="yolov4.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608

        # Load Network
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.classes = []
        self.load_class_names()
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="classes.txt"):

        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        self.colors = np.random.uniform(0, 255, size=(80, 3))
        return self.classes

    def detect(self, frame):
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)



# initialize obj deduction
od = ObjectDetection()

cap = cv2.VideoCapture("los_angeles.mp4")
#cap.set(cv2.CAP_PROP_FPS, int(60))
#print("Frame rate : {0}".format(cap.get(cv2.CAP_PROP_FPS)))

count=0
center_points_prev_frames=[]

tracking_objects={}
track_id=0


while True:
    ret, frame = cap.read()
    count+=1
    if not ret:
        break
    #points current frames
    center_points_cur_frames=[]

    # deduct obj on frames
    (class_ids, scores, boxes) = od.detect(frame)
    for box in boxes:
        (x,y,w,h)=box
        cx=int((x+x+w)/2)
        cy=int((y+y+h)/2)
        center_points_cur_frames.append((cx,cy))
        #print("FRAME N°",count," ",x,y,w,h)
        #cv2.circle(frame,(cx,cy),5,(0,0,255),-1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)


    if count<=2:
        for pt in center_points_cur_frames:
            for pt2 in center_points_prev_frames:
                distance=math.hypot(pt2[0]-pt[0],pt2[1]-pt[1])

                if distance <20:
                    tracking_objects[track_id]=pt
                    track_id+=1

    else:
        tracking_objects_copy=tracking_objects.copy()
        center_points_cur_frames_copy=center_points_cur_frames.copy()

        for object_id,pt2 in tracking_objects_copy.items():
            object_exist=False

            for pt in center_points_cur_frames_copy:
                distance=math.hypot(pt2[0]-pt[0],pt2[1]-pt[1])
                #updating object position
                if distance<20:
                    tracking_objects[object_id]=pt
                    object_exist=True
                    if pt in center_points_cur_frames:
                        center_points_cur_frames.remove(pt)
                        continue

            if not object_exist:
                tracking_objects.pop(object_id)

        for pt in center_points_cur_frames:
            tracking_objects[track_id]=pt
            track_id+=1




    for object_id,pt in tracking_objects.items():
        cv2.circle(frame,pt,5,(0,0,255),-1)
        cv2.putText(frame, str(object_id), (pt[0],pt[1] -7) ,0,1, (0,0,255), 2)



    print("Monitoring Traffic...,")
    print(tracking_objects)


    print("CURRENT FRAME LEFT PTS")
    print(center_points_cur_frames)


    cv2.imshow("Frame", frame)

    center_points_prev_frames=center_points_cur_frames.copy()



    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loading Object Detection
Running opencv dnn with YOLOv4
Monitoring Traffic...,
{}
CURRENT FRAME LEFT PTS
[(571, 891), (437, 742), (1750, 635), (761, 648), (1018, 647), (928, 533), (856, 565), (612, 473), (881, 473), (1336, 980), (753, 463), (1118, 436), (942, 458), (1877, 605), (642, 435), (688, 450), (1268, 433), (1424, 466)]


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
